In [1]:
# library that works with SHA256
import hashlib
# library that works with JSON
import json

import string
import random


In [2]:
class Block:
    
    def FormBlock(self,filename,data,StartingZeros,print = False):
        
        nonce, hash = self.FindNonce(data,StartingZeros)
        block_json = self.FormBlockJSON(data,nonce,hash)
        self.WriteTextToFile(filename,block_json)
        
        if(print):
            self.PrintBlock(block_json)
        
        return 0
    
    def VerifyBlock(self,filename,StartingZeros):
        block_json = self.ReadTextFromFile(filename)
        block_dict = json.loads(block_json)
        
        StartingZerosStr = "0" * StartingZeros
        hash_check = self.HashDataSHA256(block_dict['data'],block_dict['nonce'])
        
        if (block_dict['hash'].startswith(StartingZerosStr) and hash_check == block_dict['hash'] ):
            return True
        
        return False
    
    #
    # POW
    #
    
    def FindNonce(self,data,StartingZeros,NonceLength = 10):
        StartingZerosStr = "0" * StartingZeros
        
        # generate random str nonce with uppercase letters and digits
        nonce = ''.join(random.choices(string.ascii_uppercase + string.digits, k=NonceLength))

        dataHash = self.HashDataSHA256(data,nonce)
        
        while(not dataHash.startswith(StartingZerosStr)):
            nonce = ''.join(random.choices(string.ascii_uppercase + string.digits, k=NonceLength))
            dataHash = self.HashDataSHA256(data,nonce)
        
        return nonce, dataHash
    
    def HashDataSHA256(self,data,nonce):
        sha256 = hashlib.sha256()
        
        sha256.update((data + nonce).encode('utf-8'))
        return sha256.hexdigest()
    
    #
    #   Working with files and json
    #
        
    def FormBlockJSON(self,data,nonce,hasdata):
        
        x = {
        "data": data,
        "nonce": nonce,
        "hash": hasdata
        }   
        
        return json.dumps(x)
    
    def WriteTextToFile(self,filename,block):
        file = open(filename, "w")
        file.write(block)
        file.close()
        
    def ReadTextFromFile(self,filename):
        file = open(filename, "r")
        text = file.read()
        file.close()
        return text
    
    def PrintBlock(self,block_json):
        block_dict = json.loads(block_json)
        
        print("Block:")
        print("data: " + block_dict['data'])
        print("nonce: " + block_dict['nonce'])
        print("hash: " + block_dict['hash'])
        

In [3]:
block = Block()

data = "test message"
StartingZeroes = 5

block.FormBlock("block.txt",data,StartingZeroes,True)
valid = block.VerifyBlock("block.txt",StartingZeroes)


if(valid):
    print("\nThe block is valid.")
else:
    print("\nThe block is not valid.")



Block:
data: test message
nonce: 0M5DVW0WN9
hash: 0000036d6a60bfa6d9dc7113d08518230b0685f03d8d6bec45653fb19fa68b43

The block is valid.
